<a href="https://colab.research.google.com/github/Amanmeena2004/chatbot-/blob/main/crimebot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28

In [ ]:
!openai migrate

usage: openai [-h] [-V] [-v] [-b API_BASE] [-k API_KEY] [-p PROXY [PROXY ...]] [-o ORGANIZATION]
              {api,tools,wandb} ...
openai: error: argument {api,tools,wandb}: invalid choice: 'migrate' (choose from 'api', 'tools', 'wandb')


In [ ]:
!pip install nltk

In [ ]:
import numpy as np
import pandas as pd
import nltk
import openai
import string
import random
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
openai.api_key = 'sk-None-y311uYNkT17vlwOGjkyRT3BlbkFJfIXR5O9pHzMKlhDj6f2i'

Load the statistical data

In [ ]:
def read_csv_with_error_handling(file_path):
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None

ipc_crimes_df = read_csv_with_error_handling('/content/IPC CRIMES 2019.csv')
crime_foreigners_2022_df = read_csv_with_error_handling('/content/crimeheldwithforeigners2022.csv')
crime_foreigners_2017_2021_df = read_csv_with_error_handling('/content/crimesheldwithforeigners2017-2021.csv')

if ipc_crimes_df is not None and crime_foreigners_2022_df is not None and crime_foreigners_2017_2021_df is not None:
    nltk.download('punkt')
    nltk.download('wordnet')

    # Function to concatenate text from multiple columns
    def concatenate_columns(df, columns):
        return df[columns].astype(str).agg(' '.join, axis=1).tolist()

    # Combine text from specific columns if needed
    ipc_crimes_text = concatenate_columns(ipc_crimes_df, ['column1', 'column2'])
    crime_foreigners_2022_text = concatenate_columns(crime_foreigners_2022_df, ['column1', 'column2'])
    crime_foreigners_2017_2021_text = concatenate_columns(crime_foreigners_2017_2021_df, ['column1', 'column2'])

    # Combine all text data into a single list
    all_text = ipc_crimes_text + crime_foreigners_2022_text + crime_foreigners_2017_2021_text

    sent_tokens = nltk.sent_tokenize(" ".join(all_text))
    word_tokens = nltk.word_tokenize(" ".join(all_text))

    print(f"Sentences: {sent_tokens[:5]}")
    print(f"Words: {word_tokens[:5]}")
else:
    print("One or more CSV files could not be loaded.")

Error: [Errno 2] No such file or directory: '/content/IPC CRIMES 2019.csv'
Error: [Errno 2] No such file or directory: '/content/crimeheldwithforeigners2022.csv'
Error: [Errno 2] No such file or directory: '/content/crimesheldwithforeigners2017-2021.csv'
One or more CSV files could not be loaded.


 Combine text from all the data files into a single raw string for tokenization

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
GREET_INPUT = ("hello", "hi", "greetings", "sup", "what's up","hey")
GREET_RESPONSE = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greet(sentence):
    for word in sentence.split():
        if word.lower() in GREET_INPUT:
            return random.choice(GREET_RESPONSE)

Create and train a logistic regression model

In [ ]:
def train_model(sent_tokens):
    # Create a DataFrame with the input text data and dummy labels
    data = {
        'text': sent_tokens,
        'label': [random.choice([0, 1]) for _ in range(len(sent_tokens))]
    }
    df = pd.DataFrame(data)

    # Convert text to TF-IDF features
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    X = TfidfVec.fit_transform(df['text'])
    y = df['label']

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)

    # Train the model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Test the model
    y_pred = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

    return model, TfidfVec

model, TfidfVec = train_model(sent_tokens)

NameError: name 'sent_tokens' is not defined

In [ ]:
def response(user_response):
    robo1_response = ''
    tfidf = TfidfVec.transform([user_response])
    prediction = model.predict(tfidf)[0]
    if prediction == 0:
        robo1_response = "I am sorry! I don't understand you"
    else:
        vals = cosine_similarity(tfidf, TfidfVec.transform(sent_tokens))
        idx = vals.argsort()[0][-2]
        robo1_response = sent_tokens[idx]
    return robo1_response

In [ ]:
def chatgpt_response(prompt, max_retries=3, retry_delay=5):
    retries = 0
    while retries < max_retries:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150,
                n=1,
                stop=None,
                temperature=0.7,
            )
            return response.choices[0].message.content.strip()
        except openai.error.RateLimitError:
            retries += 1
            print(f"Rate limit exceeded. Retrying in {retry_delay} seconds... (Attempt {retries})")
            time.sleep(retry_delay)
        except Exception as e:
            print(f"An error occurred: {e}")
            break
    print("Max retries reached or error occurred. Unable to get a response.")
    return None

In [ ]:
def fetch_statistics(query):
    response = []

    # Normalize the query to lowercase
    normalized_query = query.lower()

    # Define search terms for each dataset
    search_terms = {
        'ipc crimes 2019': ipc_crimes_df,
        'crime foreigners 2022': crime_foreigners_2022_df,
        'crime foreigners 2017 2021': crime_foreigners_2017_2021_df
    }

    # Check each dataset
    for term, df in search_terms.items():
        if term in normalized_query:
            # Check if the query matches any of the rows
            for index, row in df.iterrows():
                # Use str.contains for more efficient search
                if row.astype(str).str.contains(normalized_query).any():
                    response.append(row.to_string())

    return "\n".join(response) if response else "I couldn't find any statistics related to your query."


In [ ]:
flag = True
print("ROBO: My name is Robo. Let's have a conversation! Also, if you want to exit any time, just type Bye!")

while flag:
    user_response = input()
    user_response = user_response.lower()
    if user_response != 'bye':
        if user_response == 'thanks' or user_response == 'thank you':
            flag = False
            print("ROBO: You are welcome..")
        else:
            if greet(user_response) is not None:
                print("ROBO: " + greet(user_response))
            else:
                sent_tokens.append(user_response)
                word_tokens = word_tokens + nltk.word_tokenize(user_response)
                final_words = list(set(word_tokens))
                print("ROBO: ", end="")
                bot_response = response(user_response)
                if "I am sorry! I don't understand you" in bot_response:
                    # Try to fetch statistics first
                    stats_response = fetch_statistics(user_response)
                    if stats_response == "I couldn't find any statistics related to your query.":
                        # If no statistics found, use ChatGPT response
                        bot_response = chatgpt_response(user_response)
                    else:
                        bot_response = stats_response
                print(bot_response)
                sent_tokens.remove(user_response)
    else:
        flag = False
        print("ROBO: Bye! take care..")